[Deep Learning Book](https://www.deeplearningbook.com.br/)

### Exemplo de aplicação de Redes Neurais:

- Baseado no trabalho de [Sagar Howal](https://towardsdatascience.com/neural-networks-with-google-colaboratory-artificial-intelligence-getting-started-713b5eb07f14)

- Usaremos dados a respeito de *câncer de mama*, diponíveis no [Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php), para construir uma **Rede Neural** para diagnosticar câncer de mama a partir de imagens. A base de dados contém informações descrevendo algumas características das células identificadas nas imagens. A última coluna traz o diagnóstico (2 indica **câncer benígno** e 4 indica **cancer malígno**  )
    - [Dados](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data)
    - [Descrição dos dados](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.names)

###Descrição dos dados:

- Número de instâncias: 569 

- Número de atritutos: 32 (ID, diagnóstico, 30 valores reais de entrada)

- Informação sobre os atributos:

1) ID numérico

2) Diagnóstico (M = malígno, B = benígno)

3-32) Dez valores reais de características computados para cada núcleo de célula:

	a) radius (média das distâncias do centro até um ponto no perímetro)
	b) texture (desvio padrão dos valores na escala de cinza)
	c) perimeter
	d) area
	e) smoothness (variação local no comprimento do raio)
	f) compactness (perimeter^2 / area - 1.0)
	g) concavity (severidade de porções côncavas do contorno)
	h) concave points (número de de porções côncavas do contorno)
	i) symmetry 
	j) fractal dimension ("coastline approximation" - 1)


In [ ]:
names = ['ID',
         'Diag',
         'radius1',
         'texture1', 
         'perimeter1', 
         'area1',
         'smoothness1', 
         'compactness1',
         'concavity1', 
         'concave points1',
         'symmetry1',
         'fractal dimension1',
         'radius2',
         'texture2', 
         'perimeter2', 
         'area2',
         'smoothness2', 
         'compactness2',
         'concavity2',
         'concave points2',
         'symmetry2',
         'fractal dimension2',
         'radius3',
         'texture3', 
         'perimeter3', 
         'area3',
         'smoothness3', 
         'compactness3',
         'concavity3',
         'concave points3',
         'symmetry3',
         'fractal dimension3']


In [ ]:
#Carregando o dataset - "breast_cancer.csv"
from google.colab import files
uploaded = files.upload()

In [ ]:
!ls

In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as pt
import numpy as np
import io

#Alternativamente pode-se obter o dataset do repositório
link = "http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
csv_text = requests.get(link).text
dataset = pd.read_csv(io.StringIO(csv_text))
dataset.to_csv('breast_cancer.csv')

for nome, data in uploaded.items():
  with open("breast_cancer.csv", 'wb') as f:
    f.write(data)

#Mostra as 10 primeiras linhas do dataset. 
dataset.head(10)


In [ ]:
#Inserção de rótulos para as colunas
#Se não importam os nomes das colunas, substitua "names=names" por "headers=None"
l_dataset = pd.read_csv(io.StringIO(csv_text), names=names)
l_dataset.to_csv('breast_cancer_label.csv',sep=',', encoding='utf-8')
                 
#df.to_csv('breast_cancer_lab.csv',sep='\t', encoding='utf-8')

#Mostra as 10 primeiras linhas do dataset. 
l_dataset.head(10)


In [ ]:
#from google.colab import data_table
#data_table.enable_dataframe_formatter()

#from google.colab import data_table
#data_table.disable_dataframe_formatter()

In [ ]:
!ls

In [ ]:
#Separa as variáveis dependentes da independentes. 

X = dataset.iloc[:, 2:32].values  #Nota: exclui a última coluna todos os valores NaN.
y = dataset.iloc[:, 1].values

- Os valores **M** (malígno) e **B** (benígno) devem ser convertidos para valores numérios, por ex. '1' e '0'.

In [ ]:
#Codificando os dados categóricos
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

y = labelencoder.fit_transform(y)

In [ ]:
X

In [ ]:
y

In [ ]:
#Separando o dataset em conjuntos de Treinamento (80%) e de Teste(20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


- Usaremos a **API [Keras](https://keras.io/)** para trabalharmos com Redes Neurais Artificiais
- Esta API requer **[Tensorflow](https://www.tensorflow.org/)** ou **Theano** para a execução de operações subjacentes. O Colaboratory já tem o Tensoflow instalado na máquina virtual.
- Para instalar um versão específica do Tensorflow, use: *!pip install tensorflow==XXX (XXX = versão)*
- Para instalar o Keras: *!pip install  keras*

In [ ]:
!pip show tensorflow #identica a versão do Tensorflow instalada

In [ ]:
# Importando as bibliotecas do pacote Keras
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Inicializando a ANN
classifier = Sequential()

###Especificação do parâmetros da rede neural

- **units**:  número de neurônios
- **kernel_initializer**: valores iniciais dos pesos
- **activation**: define a função de ativação

- **Primeira camada**: 5 neurônios, com pesos inicializados uniformemente e ativado pela função  ([ReLU](http://deeplearningbook.com.br/funcao-de-ativacao/)). Também definimos a dimensão do nosso *dataset* (9 colunas). Uma forma simples de determinar o número de neurônios em uma camada é dividir o número de colunas total do dataset por 2: (9+1)/2=5

- **Segunda camada**: idêntica à primeira, mas sem o parâmetro *input_dim*

- **Camada de saida**: a saída da rede tem apenas dois valores. Desta forma, basta um neurônio nesta camada. Ele terá peso uniformemente inicializado. Usaremos a função de ativação ([sigmoide](http://deeplearningbook.com.br/funcao-de-ativacao/))

In [ ]:
# Adicionando a camada de entrada e a primeira camada oculta
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 30))

# Adicionando a segunda camada oculta
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adicionando a camada de saída
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compilando a ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

- Executando:
   - **batch_size**: indica o número de entradas a serem processadas simultaneamente
   - **epoch**:  indica o  passagem  completa sobre todo o dataset.

In [ ]:
# Adequando a ANN ao conjunto de treinamento
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

In [ ]:
# Fazendo as previsões sobre o conjunto de testes
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Construindo a Matriz de Confusão
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred) 

In [ ]:
cm

**Resultados**:

- 64 negativos verdadeiros, 4 falsos positivos, 4 falsos negativos e 41 negativos verdadeiros -> **8 predições erradas**.